In [66]:
# Execute with lates qiskit to keep the time CX comparisons equal.

import time
from qiskit import transpile
import json
import os
import pickle
from qiskit import QuantumCircuit
# current_dir=os.getcwd()
# os.chdir("../Paulihedral_v2/Paulihedral_new")
# from benchmark.mypauli import pauliString
# import synthesis_FT, parallel_bl
# import importlib
# importlib.reload(synthesis_FT)
# importlib.reload(parallel_bl)
# os.chdir(current_dir)

In [67]:
#Load the pickle files.
def run_experiment_folder(folder_path = None, filename = None, save_output = False):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".pkl"):
            # Print the filename
            print(filename)
            with open(folder_path + filename, "rb") as file:
                data = pickle.load(file)
            # print(data)
            paulis=data["paulis"]
            circuit=QuantumCircuit.from_qasm_str(data["circuit"])
            results=data["results"][0]
            tot_time=results["times"]["paulihedral_time"]
            number_of_ham=results["num_paulis"]
            # Measure time for method
            start_time = time.time()
            circuit = transpile(circuit, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
            end_time = time.time()
            # include the transpilation time.
            tot_time+= (end_time - start_time)
            results = []
            # Collect results
            new_filename=filename[0:-len(".pkl")]+".json"
            result = {
                "num_paulis": number_of_ham,
                "times": {
                    "paulihedral_time": tot_time
                },
                "gate_counts": {
                    "paulihedral_method": circuit.count_ops().get('cx', 0)
                },
                "circuit_entangling_depth": {
                    "paulihedral_method": circuit.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results/test_paulihedral_' + new_filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_paulihedral_' + new_filename, 'w') as paulis_file:
                    json.dump([paulis, results], paulis_file, indent=4)
    

    

In [68]:
run_experiment_folder(folder_path="benchmarks/paulihedral_partial_results/", save_output=True)

benzene.pkl
{'num_paulis': 1255, 'times': {'paulihedral_time': 15.983575105667114}, 'gate_counts': {'paulihedral_method': 3267}, 'circuit_entangling_depth': {'paulihedral_method': 3237}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_benzene.json'}
H2O.pkl
{'num_paulis': 185, 'times': {'paulihedral_time': 1.4012553691864014}, 'gate_counts': {'paulihedral_method': 471}, 'circuit_entangling_depth': {'paulihedral_method': 461}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_H2O.json'}
labs_n10_layers1.pkl
{'num_paulis': 80, 'times': {'paulihedral_time': 0.19547533988952637}, 'gate_counts': {'paulihedral_method': 230}, 'circuit_entangling_depth': {'paulihedral_method': 209}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_labs_n10_layers1.json'}
labs_n10_layers3.pkl
{'num_paulis': 240, 'times': {'paulihedral_time': 0.5036523342132568}, 'gate_counts': {'paulihedral_method': 260}, 'circuit_entangling_depth': {'paulihedral_method': 234}, 'test_paulis_file': 'bench

In [69]:
import qiskit
print(qiskit.__version__)

1.1.1
